In [1]:
import plcoding
from plcoding.cpp_core.monotone import MCListDecoder
import numpy as np

In [2]:
# 先验概率分布
src_bases = np.array([2], dtype=int)
src_probs = np.array([0.89, 0.11])
code_lvl = 1
code_len = (1 << code_lvl)
# 生成一条简单的单调链
monotone_chain = []
for i in range(src_bases.size):
    monotone_chain.append(np.zeros((code_len,), dtype=int) + i)
monotone_chain = np.concatenate(monotone_chain)
# 初始化译码器
list_size = 4
decoder = MCListDecoder(code_len, src_bases, list_size)
#is_frozen = plcoding.topk_indicate(plcoding.bec_channels(code_lvl, e=0.5, _log=True), code_len // 2)
is_frozen = np.zeros((code_len,), dtype=bool)

In [3]:
# 生成一批测试数据
decoder.set_priors(np.tile(src_probs, (code_len, 1)))

In [4]:
decoder.reset()
headers = np.zeros(src_bases.size, dtype=int)
for sup in range(len(monotone_chain)):
    # 获取译码位置
    var = monotone_chain[i]
    index = headers[var]
    headers[var] += 1
    # 根据是否冻结，执行两种操作
    if (is_frozen[sup]):
        decoder.freeze_with(var, index, 0)
    else:
        decoder.explore_at(var, index)

In [5]:
# 最后查看一下是否正确译码
decoding_results = decoder.get_results()
decoding_results = decoding_results.reshape(-1, code_len, src_probs.size)
codewords = np.argmax(decoding_results, axis=2)

In [6]:
decoder.get_likelihoods()

array([-0.23306763, -4.41454983, -2.32380873, -2.32380873])

In [7]:
codewords

array([[0, 0],
       [1, 1],
       [0, 0],
       [1, 1]])

In [8]:
decoding_results

array([[[1., 0.],
        [1., 0.]],

       [[0., 1.],
        [0., 1.]],

       [[1., 0.],
        [1., 0.]],

       [[0., 1.],
        [0., 1.]]])